In [2]:
# pip install split-folders

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd 
import splitfolders

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam
from keras import layers
from sklearn.preprocessing import OneHotEncoder

In [3]:
#splitfolders.ratio("Data", output="Data_Split",
#    seed=42, ratio=(.64, .16, .2), group_prefix=None, move=True)

Copying files: 5856 files [00:01, 4003.95 files/s]


In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        'Data_Split/train',
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=32,
        color_mode='grayscale',
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')


Found 3747 images belonging to 2 classes.


In [10]:
validation_generator = val_datagen.flow_from_directory('Data_Split/val',
                                                        target_size=(150, 150),
                                                        batch_size=32,
                                                        color_mode='grayscale',
                                                        class_mode='binary')

Found 936 images belonging to 2 classes.


In [11]:
test_generator = test_datagen.flow_from_directory('Data_Split/test',
                                                  target_size=(150, 150),
                                                  batch_size=32,
                                                  color_mode='grayscale',
                                                  class_mode='binary')

Found 1173 images belonging to 2 classes.
